In [3]:
# Pairs Trading Model

print("=== PAIRS TRADING MODEL: LEARNING EDITION ===")
print("This version works in JupyterLite and teaches core concepts\n")

import sys
print(f"Python version: {sys.version}")

try:
    import numpy as np
    print("✅ numpy is available")
except ImportError:
    print("❌ numpy not available - this is needed for calculations")

try:
    import matplotlib.pyplot as plt
    print("✅ matplotlib is available")
except ImportError:
    print("❌ matplotlib not available - we'll describe results instead")
    plt = None

try:
    import pandas as pd
    print("✅ pandas is available")
except ImportError:
    print("❌ pandas not available - we'll use numpy arrays")
    pd = None

import math
import random
from datetime import datetime, timedelta

print("\n" + "="*60)
print("PHASE 0: Understanding Pairs Trading Fundamentals")
print("="*60)

print("What we're building:")
print("1. Two 'stocks' that move together (cointegrated)")
print("2. A way to detect when they're unusually far apart")
print("3. Trading signals based on mean reversion")
print("4. A simple backtest to see if it works")

print("\nKey mathematical concepts:")
print("- Linear regression: Finding the relationship between stocks")
print("- Residuals: The 'spread' or difference from normal relationship")
print("- Stationarity: Does the spread revert to its mean?")
print("- Z-scores: How unusual is the current spread?")


# Step 1: CREATE REALISTIC STOCK DATA


print("\n" + "="*60)
print("PHASE 1: Creating Realistic Stock Price Data")
print("="*60)

print("Since we can't download real data, let's create synthetic data that")
print("demonstrates the same mathematical principles as real stock pairs.")

# Set random seed for reproducible results
random.seed(42)
np.random.seed(42)

# Create 1000 trading days (about 4 years)
n_days = 1000
print(f"\nCreating {n_days} days of synthetic stock data...")

# Create dates
start_date = datetime(2020, 1, 1)
dates = [start_date + timedelta(days=i) for i in range(n_days)]

# Create two stocks that are cointegrated
# Think of Shell and BP - they're both oil companies, so they should move together

print("\nStock creation process:")
print("1. Common factor (oil prices) affects both stocks")
print("2. Individual factors affect each stock separately")
print("3. A mean-reverting spread ensures cointegration")

# 1. Common market factor (represents oil prices, economic conditions)
common_factor = [0]
for i in range(1, n_days):
    # Random walk with small steps (market conditions change slowly)
    change = random.gauss(0, 0.01)  # Mean=0, StdDev=0.01
    common_factor.append(common_factor[-1] + change)

# 2. Individual stock noise
shell_noise = [random.gauss(0, 0.005) for _ in range(n_days)]
bp_noise = [random.gauss(0, 0.005) for _ in range(n_days)]

# 3. Mean-reverting spread component (this ensures cointegration)
spread_component = [0]
for i in range(1, n_days):
    # Mean reversion: if spread is high, it tends to come down
    mean_reversion = -0.05 * spread_component[-1]  # 5% pull toward zero each day
    noise = random.gauss(0, 0.3)
    spread_component.append(spread_component[-1] + mean_reversion + noise)

# Construct final prices
shell_base_price = 25.0  # Shell starts around £25
bp_base_price = 5.0      # BP starts around £5

shell_prices = []
bp_prices = []

for i in range(n_days):
    # Shell price: base + common factor + individual noise + spread component
    shell_price = (shell_base_price + 
                  common_factor[i] * 3.0 +      # 3x sensitivity to common factor
                  sum(shell_noise[:i+1]) +      # Cumulative individual changes
                  spread_component[i] * 0.5)    # Half the spread component
    
    # BP price: base + common factor + individual noise - spread component
    bp_price = (bp_base_price + 
               common_factor[i] * 0.6 +         # 0.6x sensitivity to common factor
               sum(bp_noise[:i+1]) * 0.8 +      # Cumulative individual changes
               -spread_component[i] * 0.1)      # Negative spread component
    
    # Ensure prices stay positive (stocks can't go negative)
    shell_prices.append(max(shell_price, 1.0))
    bp_prices.append(max(bp_price, 0.5))

print(f"\n✅ Synthetic data created successfully!")
print(f"Shell price range: £{min(shell_prices):.2f} - £{max(shell_prices):.2f}")
print(f"BP price range: £{min(bp_prices):.2f} - £{max(bp_prices):.2f}")

# Show some sample data
print(f"\nFirst 10 days of data:")
print("Date       Shell    BP")
print("-" * 25)
for i in range(10):
    print(f"{dates[i].strftime('%Y-%m-%d')} £{shell_prices[i]:6.2f}  £{bp_prices[i]:5.2f}")


# Step 2: VISUAL ANALYSIS (WITHOUT PLOTS)


print("\n" + "="*60)
print("PHASE 2: Analyzing the Relationship")
print("="*60)

print("Let's examine if these stocks move together...")

# Calculate some basic statistics
shell_mean = sum(shell_prices) / len(shell_prices)
bp_mean = sum(bp_prices) / len(bp_prices)
shell_std = math.sqrt(sum((x - shell_mean)**2 for x in shell_prices) / len(shell_prices))
bp_std = math.sqrt(sum((x - bp_mean)**2 for x in bp_prices) / len(bp_prices))

print(f"\nBasic Statistics:")
print(f"Shell - Mean: £{shell_mean:.2f}, Std Dev: £{shell_std:.2f}")
print(f"BP    - Mean: £{bp_mean:.2f}, Std Dev: £{bp_std:.2f}")

# Calculate correlation coefficient
def correlation(x, y):
    n = len(x)
    x_mean = sum(x) / n
    y_mean = sum(y) / n
    
    numerator = sum((x[i] - x_mean) * (y[i] - y_mean) for i in range(n))
    x_var = sum((x[i] - x_mean)**2 for i in range(n))
    y_var = sum((y[i] - y_mean)**2 for i in range(n))
    
    if x_var == 0 or y_var == 0:
        return 0
    
    return numerator / math.sqrt(x_var * y_var)

corr = correlation(shell_prices, bp_prices)
print(f"\nCorrelation coefficient: {corr:.4f}")
print("Interpretation:")
if corr > 0.7:
    print("- Strong positive correlation - stocks move together! ✅")
elif corr > 0.3:
    print("- Moderate positive correlation - some relationship")
else:
    print("- Weak correlation - not ideal for pairs trading")


# Step 3: FINDING THE HEDGE RATIO (LINEAR REGRESSION)


print("\n" + "="*60)
print("PHASE 3: Finding the Mathematical Relationship")
print("="*60)

print("We want to find: Shell = α + β × BP + ε")
print("Where β (beta) is our hedge ratio")

# Simple linear regression: y = a + bx
# We want Shell = alpha + beta * BP
def linear_regression(x, y):
    n = len(x)
    x_mean = sum(x) / n
    y_mean = sum(y) / n
    
    # Calculate beta (slope)
    numerator = sum((x[i] - x_mean) * (y[i] - y_mean) for i in range(n))
    denominator = sum((x[i] - x_mean)**2 for i in range(n))
    
    if denominator == 0:
        return 0, y_mean
    
    beta = numerator / denominator
    alpha = y_mean - beta * x_mean
    
    return beta, alpha

# Run regression: Shell = alpha + beta * BP
hedge_ratio, alpha = linear_regression(bp_prices, shell_prices)

print(f"\nRegression Results:")
print(f"α (Alpha/Intercept): {alpha:.4f}")
print(f"β (Beta/Hedge Ratio): {hedge_ratio:.4f}")

print(f"\nInterpretation:")
print(f"- For every £1 that BP moves, Shell typically moves £{hedge_ratio:.4f}")
print(f"- The baseline relationship is: Shell = {alpha:.2f} + {hedge_ratio:.2f} × BP")

# Calculate R-squared (goodness of fit)
predicted_shell = [alpha + hedge_ratio * bp for bp in bp_prices]
actual_shell = shell_prices

ss_res = sum((actual_shell[i] - predicted_shell[i])**2 for i in range(len(actual_shell)))
ss_tot = sum((actual_shell[i] - shell_mean)**2 for i in range(len(actual_shell)))

r_squared = 1 - (ss_res / ss_tot) if ss_tot != 0 else 0

print(f"- R-squared: {r_squared:.4f} ({r_squared*100:.1f}% of Shell's variation explained)")


# Step 4: CALCULATE THE SPREAD


print("\n" + "="*60)
print("PHASE 4: The Spread - Our Trading Signal")
print("="*60)

print("The spread = Shell - (α + β × BP)")
print("This represents how far the stocks are from their normal relationship")

# Calculate the spread (residuals)
spread = [shell_prices[i] - (alpha + hedge_ratio * bp_prices[i]) for i in range(n_days)]

spread_mean = sum(spread) / len(spread)
spread_std = math.sqrt(sum((x - spread_mean)**2 for x in spread) / len(spread))

print(f"\nSpread Statistics:")
print(f"Mean: {spread_mean:.4f} (should be close to 0)")
print(f"Std Dev: {spread_std:.4f}")
print(f"Min: {min(spread):.4f}")
print(f"Max: {max(spread):.4f}")

print(f"\nInterpretation:")
print("- When spread > 0: Shell is expensive relative to BP")
print("- When spread < 0: Shell is cheap relative to BP")
print("- When spread ≈ 0: Stocks are at their normal relationship")


# Step 5: STATIONARITY TEST


print("\n" + "="*60)
print("PHASE 5: Testing for Mean Reversion (Simplified)")
print("="*60)

print("The Augmented Dickey-Fuller test asks:")
print("'Does this spread tend to return to its average?'")
print("\nWithout specialized libraries, we'll do a simplified test:")

# Simple test: does the spread show mean-reverting behavior?
def simple_mean_reversion_test(series, window=50):
    """
    Simple test for mean reversion:
    - Split series into chunks
    - For each chunk, see if extreme values tend to move toward the mean
    """
    mean_val = sum(series) / len(series)
    reversions = 0
    total_extreme_cases = 0
    
    for i in range(window, len(series) - 10):
        current_val = series[i]
        
        # Is current value extreme (more than 1 std dev from mean)?
        if abs(current_val - mean_val) > spread_std:
            total_extreme_cases += 1
            
            # Look at next 10 days - do we move toward the mean?
            future_vals = series[i+1:i+11]
            future_mean = sum(future_vals) / len(future_vals)
            
            # If we were above mean and moved down, or below mean and moved up
            if ((current_val > mean_val and future_mean < current_val) or
                (current_val < mean_val and future_mean > current_val)):
                reversions += 1
    
    reversion_rate = reversions / total_extreme_cases if total_extreme_cases > 0 else 0
    return reversion_rate, total_extreme_cases

reversion_rate, extreme_cases = simple_mean_reversion_test(spread)

print(f"Simple Mean Reversion Test Results:")
print(f"Extreme cases found: {extreme_cases}")
print(f"Cases that reverted toward mean: {int(reversion_rate * extreme_cases)}")
print(f"Reversion rate: {reversion_rate:.2%}")

if reversion_rate > 0.6:
    print("\n✅ Strong evidence of mean reversion - good for pairs trading!")
    is_suitable = True
elif reversion_rate > 0.4:
    print("\n⚠️ Moderate evidence of mean reversion - might work")
    is_suitable = True
else:
    print("\n❌ Weak evidence of mean reversion - not suitable for pairs trading")
    is_suitable = False

# Step 6: TRADING SIGNALS


if is_suitable:
    print("\n" + "="*60)
    print("PHASE 6: Generating Trading Signals")
    print("="*60)
    
    print("Z-score = (Current Spread - Rolling Mean) / Rolling Standard Deviation")
    print("This tells us 'how unusual is the current spread?'")
    
    # Calculate rolling statistics
    window = 30
    z_scores = []
    
    for i in range(n_days):
        if i < window:
            z_scores.append(0)  # Not enough data yet
        else:
            # Get last 30 days of spread data
            recent_spread = spread[i-window:i]
            recent_mean = sum(recent_spread) / len(recent_spread)
            recent_std = math.sqrt(sum((x - recent_mean)**2 for x in recent_spread) / len(recent_spread))
            
            if recent_std > 0:
                z_score = (spread[i] - recent_mean) / recent_std
            else:
                z_score = 0
            
            z_scores.append(z_score)
    
    # Generate trading signals
    signals = []
    for z in z_scores:
        if z > 1.5:
            signals.append(-1)  # Short the spread (sell Shell, buy BP)
        elif z < -1.5:
            signals.append(1)   # Long the spread (buy Shell, sell BP)
        else:
            signals.append(0)   # No position
    
    # Count signals
    long_signals = sum(1 for s in signals if s == 1)
    short_signals = sum(1 for s in signals if s == -1)
    
    print(f"\nSignal Summary ({window}-day rolling window):")
    print(f"Long signals (Z < -1.5): {long_signals}")
    print(f"Short signals (Z > +1.5): {short_signals}")
    print(f"Total trading opportunities: {long_signals + short_signals}")
    
    # Show some example signals
    print(f"\nExample signals (last 20 days):")
    print("Date       Spread   Z-Score  Signal")
    print("-" * 38)
    for i in range(-20, 0):
        signal_text = "LONG" if signals[i] == 1 else "SHORT" if signals[i] == -1 else "HOLD"
        print(f"{dates[i].strftime('%Y-%m-%d')} {spread[i]:7.3f} {z_scores[i]:7.2f}  {signal_text}")
    

    #Step 7: SIMPLE BACKTEST

    
    print("\n" + "="*60)
    print("PHASE 7: Simple Backtest")
    print("="*60)
    
    print("Testing our strategy:")
    print("- When Z < -1.5: Buy Shell, Sell BP (expecting spread to increase)")
    print("- When Z > +1.5: Sell Shell, Buy BP (expecting spread to decrease)")
    print("- Exit when Z crosses back near 0")
    
    # Simple backtest
    position = 0
    entry_spread = 0
    trades = []
    current_pnl = 0
    
    for i in range(1, len(z_scores)):
        current_z = z_scores[i]
        current_spread_val = spread[i]
        
        # Entry logic
        if position == 0:
            if current_z < -1.5:  # Long the spread
                position = 1
                entry_spread = current_spread_val
            elif current_z > 1.5:  # Short the spread
                position = -1
                entry_spread = current_spread_val
        
        # Exit logic
        elif position != 0:
            if abs(current_z) < 0.2:  # Exit near zero
                pnl = position * (current_spread_val - entry_spread)
                trades.append(pnl)
                current_pnl += pnl
                position = 0
    
    if trades:
        print(f"\nBacktest Results:")
        print(f"Total trades completed: {len(trades)}")
        print(f"Total P&L: {current_pnl:.4f}")
        print(f"Average P&L per trade: {current_pnl/len(trades):.4f}")
        
        winning_trades = sum(1 for t in trades if t > 0)
        win_rate = winning_trades / len(trades)
        
        print(f"Winning trades: {winning_trades}")
        print(f"Win rate: {win_rate:.2%}")
        
        print(f"\nBest trade: +{max(trades):.4f}")
        print(f"Worst trade: {min(trades):.4f}")
    else:
        print("\nNo completed trades in our test period")


# SUMMARY

print("\n" + "="*60)
print("PROJECT SUMMARY")
print("="*60)

print(f"\n🔬 Your results:")
if 'corr' in locals():
    print(f"- Stock correlation: {corr:.3f}")
if 'hedge_ratio' in locals():
    print(f"- Hedge ratio: {hedge_ratio:.3f}")
if 'r_squared' in locals():
    print(f"- Model fit (R²): {r_squared:.3f}")
if 'reversion_rate' in locals():
    print(f"- Mean reversion rate: {reversion_rate:.1%}")



=== PAIRS TRADING MODEL: LEARNING EDITION ===
This version works in JupyterLite and teaches core concepts

Python version: 3.12.7 (main, May 15 2025, 18:47:24) [Clang 19.0.0git (https:/github.com/llvm/llvm-project 0a8cd1ed1f4f35905df318015b
✅ numpy is available
✅ matplotlib is available
✅ pandas is available

PHASE 0: Understanding Pairs Trading Fundamentals
What we're building:
1. Two 'stocks' that move together (cointegrated)
2. A way to detect when they're unusually far apart
3. Trading signals based on mean reversion
4. A simple backtest to see if it works

Key mathematical concepts:
- Linear regression: Finding the relationship between stocks
- Residuals: The 'spread' or difference from normal relationship
- Stationarity: Does the spread revert to its mean?
- Z-scores: How unusual is the current spread?

PHASE 1: Creating Realistic Stock Price Data
Since we can't download real data, let's create synthetic data that
demonstrates the same mathematical principles as real stock pairs.